# Sequence-to-sequence LSTM

In [0]:
# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.__version__

'2.2.0'

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!apt install fluidsynth
!pip install midi2audio
!pip install pretty_midi

In [0]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, BatchNormalization, Dropout, LSTM, RepeatVector, TimeDistributed, Dense
import numpy as np
import pylab as plt

import sys
sys.path.insert(1, '/content/gdrive/My Drive/Music Gen/import/')
import models
import trainFac
import midiUtils

## NN Architecture

In [0]:
class Model_LSTM_seq2seq(models.Model):

    def __init__(self, input_length, output_length, output_overlap):
        super().__init__(input_length, output_length)
        self.output_overlap = output_overlap
        self.createModel()

    def generateBatches(self, path, batch_size=200, sample_spacing=1):
        generate_songs = self.infiniteSongLoop(path)
        song, idx = None, 0
        x_batch, y_batch = None, None

        i = 0
        while True:
            if x_batch is None:
                x_batch = np.zeros((batch_size, self.input_length, 48), dtype=np.float)
                y_batch = np.zeros((batch_size, self.output_length, 48), dtype=np.float)

            if song is None:
                song = next(generate_songs)
                song_t = np.transpose(song[40:88, ::])
                idx = 0

            if idx + self.input_length + self.output_length - self.output_overlap >= song.shape[1]:
                song = None
                continue         
            
            x_batch[i, ::, ::] = song_t[idx:(idx + self.input_length), ::]
            y_batch[i, ::, ::] = song_t[(idx + self.input_length - self.output_overlap):(idx + self.input_length + self.output_length - self.output_overlap), ::]
            
            idx += sample_spacing
            i += 1
            if i == batch_size:
                yield x_batch, y_batch
                x_batch, y_batch = None, None
                i = 0

    def predict(self, seed, predict_length, threshold=0.5):
        super().predict(seed, predict_length, threshold)
        assert predict_length % (self.output_length - self.output_overlap) == 0, 'Wrong predict length!'

        track = np.transpose(seed)
        result = np.copy(track)
        for _ in range(predict_length // (self.output_length - self.output_overlap)):
            new = self.model.predict(track[np.newaxis, ::, ::])
            new = new[0, -(self.output_length - self.output_overlap):, ::]
            new = (new >= threshold).astype('float')
            result = np.concatenate((result, new), axis=0)
            track = result[-self.input_length:, ::]
        return np.transpose(result)

    def createModel(self):  
        model = Sequential()

        # encoder
        model.add(Input(shape=(self.input_length, 48)))
        model.add(LSTM(256, activation='tanh', return_sequences=True))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(LSTM(256, activation='tanh'))

        # decoder
        model.add(RepeatVector(self.output_length))
        num_layers = 2
        for _ in range(num_layers):
            model.add(LSTM(256, activation='tanh', return_sequences=True))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))

        model.add(TimeDistributed(Dense(48, activation="sigmoid")))
        self.model = model
        
modelCompound = Model_LSTM_seq2seq(120, 20, 0)

# modelCompound.model.load_weights('/content/gdrive/My Drive/Music Gen/networks/lstm_seq2seq_256/train1/checkpoints/cnn-20-accuracy0.248.hdf5')
modelCompound.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 120, 256)          312320    
_________________________________________________________________
batch_normalization (BatchNo (None, 120, 256)          1024      
_________________________________________________________________
dropout (Dropout)            (None, 120, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 20, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 256)           525312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 256)           1

## Training

In [0]:
paths = trainFac.Paths(
    rootDir = '/content/gdrive/My Drive/Music Gen',
    trainPath = 'train_batches/train_7840_chorus_npz_500',
    testPath  = 'test_batches/test_800_chorus_npz_100',
    testSongPath = 'test_generation/test_songs',
    testGenerateSavePath = 'networks/lstm_seq2seq_256/train1/generate',
    checkpointBasePath = 'networks/lstm_seq2seq_256/train1/checkpoints',
    checkpointName='cnn-{epoch:02d}-accuracy{val_accuracy:.3f}.hdf5')
# create paths if they dont exist
paths.createSavePaths()

trainFacility = trainFac.TrainFacility(modelCompound, paths)
trainFacility.compile()

In [0]:
history = trainFacility.fit(epochs=20)

Epoch 1/20
10000/10000 [==============================] - ETA: 0s - loss: 0.1385 - accuracy: 0.1900
Epoch 00001: saving model to /content/gdrive/My Drive/Music Gen/networks/lstm_seq2seq_256/train1/checkpoints/cnn-01-accuracy0.190.hdf5
create save path
 -- Generating songs after epoch: 0
 -- Generating songs finished
10000/10000 [==============================] - 617s 62ms/step - loss: 0.1385 - accuracy: 0.1900 - val_loss: 0.1204 - val_accuracy: 0.1895
Epoch 2/20
10000/10000 [==============================] - ETA: 0s - loss: 0.1105 - accuracy: 0.2407
Epoch 00002: saving model to /content/gdrive/My Drive/Music Gen/networks/lstm_seq2seq_256/train1/checkpoints/cnn-02-accuracy0.216.hdf5
10000/10000 [==============================] - 599s 60ms/step - loss: 0.1105 - accuracy: 0.2407 - val_loss: 0.1208 - val_accuracy: 0.2163
Epoch 3/20
10000/10000 [==============================] - ETA: 0s - loss: 0.0963 - accuracy: 0.2753
Epoch 00003: saving model to /content/gdrive/My Drive/Music Gen/network

In [0]:
import json
with open('/content/gdrive/My Drive/Music Gen/networks/lstm_seq2seq_256/train1/history.json', 'w') as fp:
    json.dump(history.history, fp, indent=4)

## Experiments

In [0]:
net_name = 'Seq2seq'
save_path = '/content/gdrive/My Drive/Music Gen/results/seq2seq/'

In [0]:
song_name = "Can't Stop"
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/cant_stop.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Hells Bells'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/hells_bells.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Layla'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/layla.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Nothing Else Matters'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/nothing_else_matters.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Smells Like Teen Spirit'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/smells_like_teen_spirit.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Under The Bridge'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/under_the_bridge.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Unforgiven 2'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/unforgiven_2.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Unforgiven'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/unforgiven.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Wonderwall'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/wonderwall.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.